<a href="https://colab.research.google.com/github/v-enigma/DL_LabExperiments/blob/main/Lab_8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h2>Experiment 8</h2>
</center>

1.Task: Given a sequence of alphabets (with some missing values), use an RNN and a
Bidirectional RNN model to predict the missing values in the sequence.
Steps:
1. Create the dataset consisting of a sequence of alphabets.
2. Preprocess the data by encoding the alphabet characters and handling missing values.
3. Build and train an RNN model for sequence prediction.
4. Build and train a Bidirectional RNN model for comparison.
5. Predict the missing values using both models.
E.g. : M A C H I N __ predict E And using Bidirectional RNN - A C H I N E.

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Bidirectional, LSTM
from sklearn.model_selection import train_test_split

# Step 1: Create the dataset - sequences of alphabets
# Using common English words
words = [
    "MACHINE", "LEARNING", "COMPUTER", "SCIENCE", "ALGORITHM",
    "NETWORK", "PYTHON", "PROGRAM", "SEQUENCE", "DATASET",
    "TRAINING", "FUNCTION", "VARIABLE", "LANGUAGE", "GRADIENT",
    "MATRIX", "VECTOR", "TENSORFLOW", "BIDIRECTIONAL", "ACTIVATION"
]

# Step 2: Preprocess the data
# Create a mapping from characters to integers
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
char_to_int = {char: i for i, char in enumerate(alphabet)}
int_to_char = {i: char for i, char in enumerate(alphabet)}

# Create sequences and labels
sequences = []
labels = []

sequence_length = 5  # Input length - how many characters to use as input
for word in words:
    if len(word) <= sequence_length:
        continue

    for i in range(len(word) - sequence_length):
        # Input sequence
        seq = word[i:i+sequence_length]
        # Output/target is the next character
        label = word[i+sequence_length]

        sequences.append([char_to_int[char] for char in seq])
        labels.append(char_to_int[label])

# Convert to numpy arrays
X = np.array(sequences)
y = np.array(labels)

# One-hot encode the output
y_onehot = tf.keras.utils.to_categorical(y, num_classes=len(alphabet))

# Reshape input for RNN [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

# Step 3: Build and train an RNN model
def create_rnn_model():
    model = Sequential()
    model.add(SimpleRNN(128, input_shape=(X.shape[1], 1), return_sequences=False))
    model.add(Dense(len(alphabet), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

rnn_model = create_rnn_model()
print("Training RNN model...")
rnn_history = rnn_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=0)

# Step 4: Build and train a Bidirectional RNN model
def create_bidirectional_rnn_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=False), input_shape=(X.shape[1], 1)))
    model.add(Dense(len(alphabet), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

bi_rnn_model = create_bidirectional_rnn_model()
print("Training Bidirectional RNN model...")
bi_rnn_history = bi_rnn_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=0)

# Step 5: Predict the missing values
def predict_next_char(model, sequence):
    # Convert sequence to integers
    int_sequence = [char_to_int[char] for char in sequence]
    # Reshape for prediction
    x = np.array(int_sequence).reshape(1, len(sequence), 1)
    # Predict
    prediction = model.predict(x, verbose=0)
    # Get the index with highest probability
    pred_index = np.argmax(prediction)
    # Convert back to character
    return int_to_char[pred_index]

# Function to handle sequences with missing values
def predict_missing_values(model, sequence):
    """
    Predicts missing values in a sequence marked with '_'
    """
    result = list(sequence)
    missing_indices = [i for i, char in enumerate(sequence) if char == '_']

    for idx in missing_indices:
        # For each missing value, use the preceding characters to predict
        start_idx = max(0, idx - sequence_length)

        # If the missing value is at the beginning, we can't predict it using just RNN
        if idx < sequence_length:
            context = sequence[start_idx:idx]
            padding = '_' * (sequence_length - len(context))
            context = padding + context
        else:
            context = sequence[idx - sequence_length:idx]

        # Replace any missing values in context with a placeholder (we'll use 'X')
        context = [c if c != '_' else 'X' for c in context]

        pred_char = predict_next_char(model, context)
        result[idx] = pred_char

    return ''.join(result)

# Function for bidirectional prediction (handles context from both sides)
def predict_missing_values_bidirectional(sequence):
    """
    For bidirectional RNN, we simulate bidirectional context by using both models
    and taking the most confident prediction. In a real implementation, we'd use
    a custom training approach for the missing value task.
    """
    result = list(sequence)
    missing_indices = [i for i, char in enumerate(sequence) if char == '_']

    for idx in missing_indices:
        # First, check if we have enough context on both sides
        has_left = idx >= sequence_length
        has_right = len(sequence) - idx - 1 >= sequence_length

        if has_left:
            left_context = sequence[idx - sequence_length:idx]
            left_context = [c if c != '_' else 'X' for c in left_context]
            left_pred = predict_next_char(rnn_model, left_context)
        else:
            left_pred = None

        if has_right:
            # For right context, we reverse the sequence and use it
            right_context = sequence[idx + 1:idx + sequence_length + 1]
            right_context = [c if c != '_' else 'X' for c in right_context]
            right_context = right_context[::-1]  # reverse
            right_pred = predict_next_char(rnn_model, right_context)
        else:
            right_pred = None

        # Use bidirectional model's prediction as the final decision
        if left_pred and right_pred:
            # In reality, we would use the actual bidirectional RNN output here
            pred_char = predict_next_char(bi_rnn_model,left_context + [right_context[0]])  # Just a proxy
        elif left_pred:
            pred_char = left_pred
        elif right_pred:
            pred_char = right_pred
        else:
            # If no context, just guess 'A'
            pred_char = 'A'

        result[idx] = pred_char

    return ''.join(result)

# Example of prediction
example1 = "MACHIN_"
example2 = "_ACHINE"
example3 = "MAC_INE"

print("\nPredictions:")
print(f"Original (with missing): {example1}")
print(f"RNN prediction: {predict_missing_values(rnn_model, example1)}")
print(f"Bidirectional prediction: {predict_missing_values_bidirectional(example1)}")

print(f"\nOriginal (with missing): {example2}")
print(f"RNN prediction: {predict_missing_values(rnn_model, example2)}")
print(f"Bidirectional prediction: {predict_missing_values_bidirectional(example2)}")

print(f"\nOriginal (with missing): {example3}")
print(f"RNN prediction: {predict_missing_values(rnn_model, example3)}")
print(f"Bidirectional prediction: {predict_missing_values_bidirectional(example3)}")

# Test using the examples from the problem statement
print("\nProblem examples:")
example_forward = "M A C H I N _"  # Expected: E
example_forward = example_forward.replace(" ", "")  # Remove spaces
print(f"Forward prediction (RNN): {predict_missing_values(rnn_model, example_forward)}")

example_backward = "_ C H I N E"  # Expected: A
example_backward = example_backward.replace(" ", "")  # Remove spaces
print(f"Using Bidirectional: {predict_missing_values_bidirectional(example_backward)}")

# Performance metrics
rnn_loss, rnn_acc = rnn_model.evaluate(X_test, y_test, verbose=0)
bi_rnn_loss, bi_rnn_acc = bi_rnn_model.evaluate(X_test, y_test, verbose=0)

print("\nModel Performance:")
print(f"RNN Test Accuracy: {rnn_acc:.4f}")
print(f"Bidirectional RNN Test Accuracy: {bi_rnn_acc:.4f}")

Training RNN model...
Training Bidirectional RNN model...

Predictions:
Original (with missing): MACHIN_
RNN prediction: MACHINE
Bidirectional prediction: MACHINE

Original (with missing): _ACHINE
RNN prediction: FACHINE
Bidirectional prediction: EACHINE

Original (with missing): MAC_INE
RNN prediction: MACLINE
Bidirectional prediction: MACAINE

Problem examples:
Forward prediction (RNN): MACHINE
Using Bidirectional: ECHINE

Model Performance:
RNN Test Accuracy: 0.4167
Bidirectional RNN Test Accuracy: 0.0833
